In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1616478/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.7)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_3_40,0.809274,0.255276,0.547706,0.360304,0.471675,0.030706,0.442100,0.080347,0.099736,0.090041,0.055474,0.175232,0.602654,0.176640,136.966571,321.324895,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_3_44,0.809226,0.236363,0.538974,0.355782,0.465055,0.030714,0.453328,0.081898,0.100441,0.091170,0.054805,0.175254,0.602553,0.176662,136.966066,321.324390,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_3_39,0.809021,0.264684,0.548096,0.369456,0.476064,0.030747,0.436516,0.080278,0.098309,0.089293,0.055881,0.175348,0.602127,0.176757,136.963923,321.322247,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_3_41,0.808671,0.253770,0.549317,0.354107,0.469679,0.030803,0.442995,0.080061,0.100702,0.090381,0.055548,0.175509,0.601399,0.176918,136.960265,321.318589,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_3_38,0.808419,0.285648,0.539955,0.398371,0.485048,0.030844,0.424070,0.081724,0.093801,0.087762,0.057284,0.175624,0.600873,0.177035,136.957630,321.315954,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,model_1_3_1,-0.565462,-0.126053,-2.571550,0.083010,-1.280819,0.252035,0.668474,0.634462,0.142969,0.388715,0.110297,0.502031,-2.261380,0.506063,132.756377,317.114701,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
246,model_1_1_46,-0.578874,-0.312412,-6.716027,-6.612432,-6.666255,0.254194,0.779105,0.925189,0.732445,0.828817,0.082094,0.504177,-2.289321,0.508226,132.739315,317.097639,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
247,model_1_1_47,-0.581993,-0.318312,-6.680614,-6.691280,-6.681704,0.254696,0.782607,0.920943,0.740032,0.830487,0.082105,0.504674,-2.295818,0.508728,132.735369,317.093692,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
248,model_1_1_48,-0.645470,-0.384519,-7.061820,-6.922790,-6.996117,0.264916,0.821910,0.966651,0.762307,0.864479,0.082962,0.514700,-2.428062,0.518834,132.656687,317.015011,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
